# Combining COMTRADE and Flaring data

GGFR have a global flare datasets derived from Elvidge at point locations. They further map these point locations to operators via oil field mapping. This means there are a number of datasets:

1. Point locations of flaring
2. Mapped oil field locations (from Woods Mackenzie)
3. Tabulated national accounts (of flaring)

Harshit suggested there could be an interesting combination of the COMTRADE data to see what the comparison of nationally generated flaring vs imported flaring.

Flaring Supply Index (FSI) is a concept proposed for an indicator that would be able to describe and compare the ’flaring footprint’ of different oil supply corridors from their source to the destination country’s border. The formula(s) for such an index may be described as follows.
Flare Supply Index of a Country X= (∑_(i=1)^n▒〖Crude Import by X from Country Yi * Flare Intensity of Country Yi〗)/(Total Crude Imported by Country X)


## Definitions
Flare Intensity - ???  
TOE - Tonnes of oil equivalent

## TO DO
1. TOE is not the appropriate flaring comparison for intensity, figure out what it is
2. Right now, the intensity is taken just from 2019; should use yearly data

In [1]:
import sys, os, importlib
import json, geojson, pycountry

import pandas as pd
import geopandas as gpd
import numpy as np

In [21]:
# Define input variables
# COMTRADE data can be downloaded here - https://datacatalog.worldbank.org/dataset/global-comtrade-flows-data
comtrade_file = '/home/wb411133/data/Projects/INFRA/FLOWS/Oil/GEOJSON/oil_country_flows_imports.geojson'
flare_data = '/home/public/Data/GLOBAL/INFRA/FLARING/2019_flare_catalog.csv'
flare_national_data = '/home/public/Data/GLOBAL/INFRA/FLARING/National_flaring_2019.csv'


inCom = gpd.read_file(comtrade_file)
inFlare = pd.read_csv(flare_data)
inF = pd.read_csv(flare_national_data)

The location specific flare data is not necessary for calculating the FSI, but it could be used in other calculations

In [22]:
inCom.head()

,Reporter ISO,Partner ISO,Trade Flow,Year,Qty,Trade Value (US$),TOE,Reporter_Pt,Partner_Pt,Value per unit,geometry
0,,,Import,2017,2.534500e+05,3.064940e+05,3.100445e+02,None,None,1.209288,None
1,,AGO,Import,2017,3.528180e+09,1.466747e+09,4.316011e+06,None,POINT (17.54847831062573 -12.29583346306004),0.415723,None
2,,ARE,Import,2017,7.335661e+09,3.219519e+09,8.973690e+06,None,POINT (54.32869156271079 23.90682311096173),0.438886,None
3,,AUS,Import,2017,1.065144e+08,5.844422e+07,1.302987e+05,None,POINT (134.4870620118277 -25.73268264712671),0.548698,None
4,,AUT,Import,2017,2.845370e+05,5.179050e+05,3.480732e+02,None,POINT (14.14002973484645 47.58852296473773),1.820168,None


In [23]:
inCom.loc[(inCom['Reporter ISO'] == "JPN") & (inCom['Year'] == '2017')].sort_values('TOE', ascending=False)

,Reporter ISO,Partner ISO,Trade Flow,Year,Qty,Trade Value (US$),TOE,Reporter_Pt,Partner_Pt,Value per unit,geometry
6030,JPN,SAU,Import,2017,6.507555e+10,2.642928e+10,7.960670e+07,POINT (137.9632416223659 37.54022267074495),POINT (44.54372307264682 24.12390168405157),0.406132,LINESTRING (44.54372307264682 24.1239016840515...
5951,JPN,ARE,Import,2017,4.127565e+10,1.737742e+10,5.049237e+07,POINT (137.9632416223659 37.54022267074495),POINT (54.32869156271079 23.90682311096173),0.421009,LINESTRING (54.32869156271079 23.9068231109617...
6028,JPN,QAT,Import,2017,1.471252e+10,6.279196e+09,1.799778e+07,POINT (137.9632416223659 37.54022267074495),POINT (51.19868832434384 25.28053193101709),0.426793,LINESTRING (51.19868832434384 25.2805319310170...
6002,JPN,KWT,Import,2017,1.356020e+10,5.384218e+09,1.658815e+07,POINT (137.9632416223659 37.54022267074495),POINT (47.61290724370362 29.35199533223659),0.397060,LINESTRING (47.61290724370362 29.3519953322365...
6029,JPN,RUS,Import,2017,1.105409e+10,4.731385e+09,1.352243e+07,POINT (137.9632416223659 37.54022267074495),POINT (96.59005059203922 61.95356043548515),0.428021,LINESTRING (96.59005059203922 61.9535604354851...
5995,JPN,IRN,Import,2017,8.744804e+09,3.524027e+09,1.069749e+07,POINT (137.9632416223659 37.54022267074495),POINT (54.19893299230245 32.73732737127096),0.402985,LINESTRING (54.19893299230245 32.7373273712709...
6046,JPN,USA,Import,2017,6.710220e+09,1.358344e+09,8.208590e+06,POINT (137.9632416223659 37.54022267074495),POINT (-96.05062671003752 37.04258440408181),0.202429,LINESTRING (-96.05062671003752 37.042584404081...
6001,JPN,KOR,Import,2017,5.931195e+09,3.074745e+09,7.255610e+06,POINT (137.9632416223659 37.54022267074495),POINT (127.8063292399667 36.35670136710129),0.518402,LINESTRING (127.8063292399667 36.3567013671012...
5996,JPN,IRQ,Import,2017,2.704910e+09,1.008425e+09,3.308907e+06,POINT (137.9632416223659 37.54022267074495),POINT (43.77473247071985 33.04588584276406),0.372813,LINESTRING (43.77473247071985 33.0458858427640...
5993,JPN,IND,Import,2017,2.496343e+09,1.201940e+09,3.053768e+06,POINT (137.9632416223659 37.54022267074495),POINT (79.28876794161593 22.74980155745747),0.481480,LINESTRING (79.28876794161593 22.7498015574574...


In [24]:
agg = {'bcm':'sum'}
flare_combo = inFlare.groupby(['iso']).aggregate(agg).reset_index()
flare_combo.head()

,iso,bcm
0,AGO,2.325094
1,ALB,0.002380
2,ARE,1.027568
3,ARG,1.029591
4,AUS,1.405146


In [25]:
# Limit the flaring summaries to 2019
flare_2019 = inF.filter(regex="2019")
flare_2019.columns = ["Volume","Intensity"]
flare_2019['country'] = inF['country']
flare_2019.head()

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Volume,Intensity,country
0,23212.170060,5.862699884,Russia
1,17914.219000,10.37717386,Iraq
2,17293.742520,3.868390139,United States
3,13781.155540,12.59168227,Iran
4,9541.420483,29.81330551,Venezuela


In [26]:
def get_country(x):
    ''' Convert country name to ISO3
    
    :param: x [string] - name of country to convert
    :returns: [string] - ISO3 code of country
    '''
    try:
        res = pycountry.countries.search_fuzzy(x)
        return(res[0].alpha_3)
    except:
        return("NA")

#xx = get_country("Canada")
flare_2019['ISO3'] = flare_2019['country'].apply(lambda x: get_country(x))
flare_2019.head()

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Volume,Intensity,country,ISO3
0,23212.170060,5.862699884,Russia,RUS
1,17914.219000,10.37717386,Iraq,IRQ
2,17293.742520,3.868390139,United States,USA
3,13781.155540,12.59168227,Iran,IRN
4,9541.420483,29.81330551,Venezuela,VEN


In [34]:
# 5 of the country names don't grok, set them manually

#flare_2019.loc[flare_2019['ISO3'] == "NA"]
flare_2019.iloc[16, 3] = "COG"
flare_2019.iloc[30, 3] = "ARE"
flare_2019.iloc[44, 3] = "COD"
flare_2019.iloc[60, 3] = "NER"
flare_2019.loc[flare_2019['ISO3'] == "NA"]

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Volume,Intensity,country,ISO3


In [35]:
#Combine the flare intensity information with the comtrade data
in_flaring_combined = pd.merge(inCom, flare_2019, left_on="Partner ISO", right_on="ISO3", how="left")
#Drop unnecessary columns
in_flaring_combined = in_flaring_combined.drop(['Reporter_Pt', 'Partner_Pt', 'geometry', 'country', 'ISO3'], axis=1)

def tryFloat(x):
    try:
        return(float(x))
    except:
        return(0.)

# Multiply the TOE from the comtrade data by the intensity information
### TODO: TOE is not the unit appropriate for intensity calculations; however, the relative rank is still useful
in_flaring_combined['TOE'] = in_flaring_combined['TOE'].apply(float)
in_flaring_combined['Intensity'] = in_flaring_combined['Intensity'].apply(tryFloat)
in_flaring_combined['TOE_I'] = in_flaring_combined['TOE'] * in_flaring_combined['Intensity']
in_flaring_combined.head()

,Reporter ISO,Partner ISO,Trade Flow,Year,Qty,Trade Value (US$),TOE,Value per unit,Volume,Intensity,TOE_I
0,,,Import,2017,2.534500e+05,3.064940e+05,3.100445e+02,1.209288,NaN,NaN,NaN
1,,AGO,Import,2017,3.528180e+09,1.466747e+09,4.316011e+06,0.415723,2325.103979,4.400772,1.899378e+07
2,,ARE,Import,2017,7.335661e+09,3.219519e+09,8.973690e+06,0.438886,900.272261,0.721855,6.477703e+06
3,,AUS,Import,2017,1.065144e+08,5.844422e+07,1.302987e+05,0.548698,1389.791288,10.554468,1.375233e+06
4,,AUT,Import,2017,2.845370e+05,5.179050e+05,3.480732e+02,1.820168,NaN,NaN,NaN


In [37]:
in_flaring_combined.loc[(in_flaring_combined['Reporter ISO'] == "JPN") & (in_flaring_combined['Year'] == "2017")].sort_values('TOE', ascending=False).head()

,Reporter ISO,Partner ISO,Trade Flow,Year,Qty,Trade Value (US$),TOE,Value per unit,Volume,Intensity,TOE_I
6030,JPN,SAU,Import,2017,6.507555e+10,2.642928e+10,79606700.9,0.406132,2100.455544,0.585661,4.662258e+07
5951,JPN,ARE,Import,2017,4.127565e+10,1.737742e+10,50492370.5,0.421009,900.272261,0.721855,3.644817e+07
6028,JPN,QAT,Import,2017,1.471252e+10,6.279196e+09,17997779.0,0.426793,1343.748416,2.422041,4.359136e+07
6002,JPN,KWT,Import,2017,1.356020e+10,5.384218e+09,16588149.5,0.397060,730.990357,0.692833,1.149281e+07
6029,JPN,RUS,Import,2017,1.105409e+10,4.731385e+09,13522429.5,0.428021,23212.170060,5.862700,7.927795e+07


In [38]:
in_flaring_combined.to_csv("FLARING_TRADE_COMBINED_ALLTRADE.csv")

In [ ]:
# Aggregate imported TOE, Value and TOE by import intensity
agg = {'TOE':'sum','Trade Value (US$)':'sum', 'Intensity':'mean','TOE_I':'sum'}
res = in_flaring_combined.groupby(['Reporter ISO', "Year"]).aggregate(agg)
res = res.reset_index()
res.head(20)

In [ ]:
res.to_csv("NATIONAL_FIS_v0.csv")

In [45]:
in_flaring_combined.loc[(in_flaring_combined['Reporter ISO'] == "JPN")].sort_values("Partner ISO", ascending=True)

,Reporter ISO,Partner ISO,Trade Flow,Year,Qty,Trade Value (US$),TOE,Value per unit,Volume,Intensity,TOE_I
5948,JPN,,Import,2017,6.845832e+07,3.783837e+07,8.374484e+04,0.552721,NaN,NaN,NaN
5949,JPN,AGO,Import,2017,5.238670e+08,2.033851e+08,6.408448e+05,0.388238,2325.103979,4.400772,2.820212e+06
5950,JPN,ANT,Import,2010,1.092000e+05,2.094000e+05,1.335840e+02,1.917582,NaN,NaN,NaN
5951,JPN,ARE,Import,2017,4.127565e+10,1.737742e+10,5.049237e+07,0.421009,900.272261,0.721855,3.644817e+07
5952,JPN,ARG,Import,2015,1.134000e+03,7.677000e+03,1.387218e+00,6.769841,941.830117,5.087041,7.056838e+00
5953,JPN,ATF,Import,2017,6.000000e+02,1.390100e+04,7.339780e-01,23.168333,NaN,NaN,NaN
5954,JPN,AUS,Import,2017,6.323227e+08,2.978807e+08,7.735183e+05,0.471090,1389.791288,10.554468,8.164074e+06
5955,JPN,AUT,Import,2017,1.271110e+05,5.187300e+05,1.554945e+02,4.080921,NaN,NaN,NaN
5956,JPN,AZE,Import,2012,2.670642e+07,2.369838e+07,3.266988e+04,0.887366,222.731902,0.795309,2.598265e+04
5957,JPN,BEL,Import,2017,1.110985e+07,1.483004e+07,1.359065e+04,1.334855,NaN,NaN,NaN
